# メソッドの可視化

* publicが指定されたメソッド
    * どのインスタンスからも実行できる
    * デフォルト
* protectedが指定されたメソッド
    * 自分自身は実行できる
    * サブクラスのインスタンスから実行できる
* privateが指定されたメソッド
    * <font color="red">レシーバをつけた呼び出しはできない</font>
    * [参考] Javaの場合
        * クラス内のみアクセス可能
        
        
protectedとprivateの違いはそのレシーバを自クラスかサブクラスのレシーバーが利用するかどうかの違い

[Ruby の private と protected 。歴史と使い分け](http://qiita.com/tbpgr/items/6f1c0c7b77218f74c63e)

In [38]:
class Foo
  def public_method1
    "public method1"
  end

  ### デフォルトがpublicなので何も変わらない
  public

  def public_method2
    "public method2"
  end
  
  def methodA
    protected_method1
  end

  def methodC
    private_method1
  end

  protected

  def protected_method1
    "protected method1"
  end
  def protected_method2
    "protected method2"
  end
  
  private

  def private_method1
    "private method1"
  end
  def private_method2
    "private method2"
  end
end

class FooExt < Foo
  def methodB
    protected_method2
  end

  def methodD
    private_method2
  end
end

puts "### publicメソッドの実行"
puts Foo.new.public_method1
puts Foo.new.public_method2
puts ""

puts "### 自分自身(Foo)のメソッド(methodA)からprotectedが指定されたメソッド(protected_method1)を実行"
puts Foo.new.methodA
puts "### サブクラス(FooExt)のメソッド(methodB)からprotectedが指定されたメソッド(protected_method2)を実行"
puts FooExt.new.methodB
### メソッドを介せず直接実行するとNoMethodError
# puts Foo.new.protected_method1
puts ""

puts "### 自分自身(Foo)のメソッド(methodC)からprivateが指定されたメソッド(private_method1)を実行"
puts Foo.new.methodC
puts "### サブクラス(FooExt)のメソッド(methodD)からprivateが指定されたメソッド(private_method2)を実行"
puts FooExt.new.methodD
### メソッドを介せず直接実行するとNoMethodError
#puts Foo.new.private_method1

### publicメソッドの実行
public method1
public method2

### 自分自身(Foo)のメソッド(methodA)からprotectedが指定されたメソッド(protected_method1)を実行
protected method1
### サブクラス(FooExt)のメソッド(methodB)からprotectedが指定されたメソッド(protected_method2)を実行
protected method2

### 自分自身(Foo)のメソッド(methodC)からprivateが指定されたメソッド(private_method1)を実行
private method1
### サブクラス(FooExt)のメソッド(methodD)からprivateが指定されたメソッド(private_method2)を実行
private method2


## 引数を指定した呼び出し

In [30]:
class Foo2
  def public_method1; 1; end
  def public_method2; 2; end
  def protected_method1; 1; end
  def protected_method2; 2; end
  def private_method1; 1; end
  
  ### メソッド名をシンボルで渡して後で指定
  public :public_method1, :public_method2
  protected :protected_method1, :protected_method2
  private :private_method1
end

Foo2

## privateの振る舞い

* selfなどレシーバを指定するとエラーになる
    * protectedはならない

In [1]:
### privateメソッドの挙動
class Foo3
  def public_method1
    private_method1
  end
  
  def public_method2
    self.private_method1
  end

  private
  
  def private_method1; "private method"; end
end

puts Foo3.new.public_method1
### selfなどレシーバを指定するとエラーとなる
#Foo3.new.public_method2

private method


In [37]:
### protectedメソッドの挙動
class Foo4
  def public_method1
    protected_method1
  end
  
  def public_method2
    self.protected_method1
  end

  protected
  
  def protected_method1; "protected method"; end
end

puts Foo4.new.public_method1
### selfなどレシーバを指定してもエラーにならない
puts Foo4.new.public_method2

protected method
protected method


In [17]:
class Foo5
  def use_protected_method(other_obj)
    other_obj.protected_method
  end

  def use_private_method(other_obj)
    other_obj.private_method   ### privateメソッドはレシーバを明示的に指定できないのでエラーになる
  end

  protected
  
  def protected_method; "protected method"; end

  private
  
  def private_method; "private method"; end
end

x = Foo5.new
y = Foo5.new

puts x.use_protected_method(y)

begin
  puts x.use_private_method(y)  ### オブジェクトxは同じクラスのオブジェクトyのプライベートメソッドを呼び出せない
rescue => ex
  puts ex
end

protected method
private method `private_method' called for #<Foo5:0x007f9f9d9fcb00>


In [13]:
class Foo2
  def use_private_method(other_obj)
    other_obj.private_method
  end

  private
  
  def private_method; "private method"; end
end

x = Foo2.new
y = Foo2.new

putes x.use_private_method(y)

NoMethodError: private method `private_method' called for #<Foo2:0x007f9f9c9ba580>

## サブクラスで可視化の変更

In [41]:
class Foo
  private 
  def private_method1
    "private method1"
  end
end

class FooExt < Foo
  ### publicに可視化を変更
  public :private_method1
end

### privateメソッドは直接実行できない
#Foo.new.private_method1

### サブクラスで可視化をpublicに変更しているので直接実行できる
FooExt.new.private_method1

"private method1"

# Kernelモジュールと関数

* Kernelモジュールのメソッド
    * puts
    * proc
    * その他
* Rubyは本当の意味での関数は存在しない
    * 関数に見えるものはすべてメソッド

## Kernelモジュールの呼ばれ方

* トップレベルのクラスはObject
* <font color="red">ObjectクラスはKernelモジュールをincludeしている</font>
    * 最終的にKernelモジュールで定義されたメソッドが呼び出される
* Objectクラスは全てのクラスのスーパークラス
    * Kernelモジュールに定義したメソッドはプログラム中のどこからでも呼び出すことができる
    * Kernelモジュールの多くのメソッドはprivateが指定されているのでレシーバを指定して呼び出すことはできない
    * Rubyの組み込み関数と同等のものを定義したい場合は、Kernelモジュールにprivateメソッドを定義すればよい
        * ただし、<font color="red">特に理由がない限りはKernelモジュールよりもObjectクラスにprivateメソッドを定義することが推奨される</font>

In [52]:
### トップレベルのクラスはObject
puts Object.instance_methods
puts self.class
puts #

### レシーバを指定してputsメソッドを呼ぶことはできない
puts Object.private_methods
#self.puts

[:pry, :__binding__, :pretty_print, :pretty_print_cycle, :pretty_print_instance_variables, :pretty_print_inspect, :to_json, :instance_of?, :public_send, :instance_variable_get, :instance_variable_set, :instance_variable_defined?, :remove_instance_variable, :private_methods, :kind_of?, :instance_variables, :tap, :public_method, :singleton_method, :is_a?, :extend, :define_singleton_method, :method, :to_enum, :enum_for, :<=>, :===, :=~, :!~, :eql?, :respond_to?, :freeze, :inspect, :display, :object_id, :send, :gem, :to_s, :pretty_inspect, :nil?, :hash, :class, :singleton_class, :clone, :dup, :itself, :taint, :tainted?, :untaint, :untrust, :trust, :untrusted?, :methods, :protected_methods, :frozen?, :public_methods, :singleton_methods, :!, :==, :!=, :__send__, :equal?, :instance_eval, :instance_exec, :__id__]
Object

[:initialize, :inherited, :attr_reader, :attr, :attr_accessor, :attr_writer, :remove_const, :undef_method, :method_added, :method_removed, :remove_method, :method_undefined, :

In [57]:
### Kernelモジュールにprivateメソッドを追加
module Kernel
  private
  def my_func; 1; end
end

puts my_func
### Error
#puts self.my_func

### Objectクラスにprivateメソッドを追加すること推奨
class Object
  private
  def my_func2; 2; end
end

puts my_func2
### Error
#puts self.my_func2

1
2


### selfについて

* 4-4.SingletonClass.ipynb で記載